# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 16 2022 12:00PM,244271,19,60017833.1,"GUSA Granules USA, Inc.",Released
1,Aug 16 2022 11:54AM,244269,10,MTR-22-0244-BO,"Senseonics, Incorporated",Released
2,Aug 16 2022 11:54AM,244270,19,60016351,"GUSA Granules USA, Inc.",Released
3,Aug 16 2022 11:52AM,244267,10,0085939045,ISDIN Corporation,Released
4,Aug 16 2022 11:52AM,244267,10,0085939046,ISDIN Corporation,Released
5,Aug 16 2022 11:52AM,244267,10,0085939047,ISDIN Corporation,Released
6,Aug 16 2022 11:52AM,244267,10,0085939048,ISDIN Corporation,Released
7,Aug 16 2022 11:52AM,244267,10,0085939049,ISDIN Corporation,Released
8,Aug 16 2022 11:52AM,244267,10,0085939105,ISDIN Corporation,Released
9,Aug 16 2022 11:52AM,244267,10,0085939125,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,244267,Released,18
42,244268,Released,1
43,244269,Released,1
44,244270,Released,1
45,244271,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244267,NaN,NaN,18.0
244268,NaN,NaN,1.0
244269,NaN,NaN,1.0
244270,NaN,NaN,1.0
244271,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244164,0.0,0.0,1.0
244183,0.0,0.0,1.0
244189,0.0,0.0,1.0
244191,0.0,0.0,1.0
244192,0.0,0.0,12.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244164,0,0,1
244183,0,0,1
244189,0,0,1
244191,0,0,1
244192,0,0,12


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244164,0,0,1
1,244183,0,0,1
2,244189,0,0,1
3,244191,0,0,1
4,244192,0,0,12


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244164,,,1
1,244183,,,1
2,244189,,,1
3,244191,,,1
4,244192,,,12


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 16 2022 12:00PM,244271,19,"GUSA Granules USA, Inc."
1,Aug 16 2022 11:54AM,244269,10,"Senseonics, Incorporated"
2,Aug 16 2022 11:54AM,244270,19,"GUSA Granules USA, Inc."
3,Aug 16 2022 11:52AM,244267,10,ISDIN Corporation
21,Aug 16 2022 11:52AM,244268,20,"ACI Healthcare USA, Inc."
22,Aug 16 2022 11:50AM,244266,22,"NBTY Global, Inc."
23,Aug 16 2022 11:46AM,244265,10,ISDIN Corporation
29,Aug 16 2022 11:45AM,244264,10,Emerginnova
31,Aug 16 2022 11:44AM,244263,10,"Snap Medical Industries, LLC"
33,Aug 16 2022 11:42AM,244262,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 16 2022 12:00PM,244271,19,"GUSA Granules USA, Inc.",,,1
1,Aug 16 2022 11:54AM,244269,10,"Senseonics, Incorporated",,,1
2,Aug 16 2022 11:54AM,244270,19,"GUSA Granules USA, Inc.",,,1
3,Aug 16 2022 11:52AM,244267,10,ISDIN Corporation,,,18
4,Aug 16 2022 11:52AM,244268,20,"ACI Healthcare USA, Inc.",,,1
5,Aug 16 2022 11:50AM,244266,22,"NBTY Global, Inc.",,,1
6,Aug 16 2022 11:46AM,244265,10,ISDIN Corporation,,,6
7,Aug 16 2022 11:45AM,244264,10,Emerginnova,,,2
8,Aug 16 2022 11:44AM,244263,10,"Snap Medical Industries, LLC",,,2
9,Aug 16 2022 11:42AM,244262,19,ACG North America LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 12:00PM,244271,19,"GUSA Granules USA, Inc.",1,,
1,Aug 16 2022 11:54AM,244269,10,"Senseonics, Incorporated",1,,
2,Aug 16 2022 11:54AM,244270,19,"GUSA Granules USA, Inc.",1,,
3,Aug 16 2022 11:52AM,244267,10,ISDIN Corporation,18,,
4,Aug 16 2022 11:52AM,244268,20,"ACI Healthcare USA, Inc.",1,,
5,Aug 16 2022 11:50AM,244266,22,"NBTY Global, Inc.",1,,
6,Aug 16 2022 11:46AM,244265,10,ISDIN Corporation,6,,
7,Aug 16 2022 11:45AM,244264,10,Emerginnova,2,,
8,Aug 16 2022 11:44AM,244263,10,"Snap Medical Industries, LLC",2,,
9,Aug 16 2022 11:42AM,244262,19,ACG North America LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 12:00PM,244271,19,"GUSA Granules USA, Inc.",1,,
1,Aug 16 2022 11:54AM,244269,10,"Senseonics, Incorporated",1,,
2,Aug 16 2022 11:54AM,244270,19,"GUSA Granules USA, Inc.",1,,
3,Aug 16 2022 11:52AM,244267,10,ISDIN Corporation,18,,
4,Aug 16 2022 11:52AM,244268,20,"ACI Healthcare USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 12:00PM,244271,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
1,Aug 16 2022 11:54AM,244269,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Aug 16 2022 11:54AM,244270,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
3,Aug 16 2022 11:52AM,244267,10,ISDIN Corporation,18.0,NaN,NaN
4,Aug 16 2022 11:52AM,244268,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 12:00PM,244271,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
1,Aug 16 2022 11:54AM,244269,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Aug 16 2022 11:54AM,244270,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
3,Aug 16 2022 11:52AM,244267,10,ISDIN Corporation,18.0,0.0,0.0
4,Aug 16 2022 11:52AM,244268,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3663699,138.0,0.0,0.0
15,732653,60.0,62.0,2.0
16,1221058,4.0,1.0,0.0
19,1709690,6.0,2.0,0.0
20,1465437,61.0,0.0,0.0
21,732583,3.0,0.0,0.0
22,732772,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3663699,138.0,0.0,0.0
1,15,732653,60.0,62.0,2.0
2,16,1221058,4.0,1.0,0.0
3,19,1709690,6.0,2.0,0.0
4,20,1465437,61.0,0.0,0.0
5,21,732583,3.0,0.0,0.0
6,22,732772,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,138.0,0.0,0.0
1,15,60.0,62.0,2.0
2,16,4.0,1.0,0.0
3,19,6.0,2.0,0.0
4,20,61.0,0.0,0.0
5,21,3.0,0.0,0.0
6,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,138.0
1,15,Released,60.0
2,16,Released,4.0
3,19,Released,6.0
4,20,Released,61.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,0.0,2.0,0.0,0.0,0.0,0.0,0.0
Executing,0.0,62.0,1.0,2.0,0.0,0.0,0.0
Released,138.0,60.0,4.0,6.0,61.0,3.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,0.0,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,62.0,1.0,2.0,0.0,0.0,0.0
2,Released,138.0,60.0,4.0,6.0,61.0,3.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,0.0,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,62.0,1.0,2.0,0.0,0.0,0.0
2,Released,138.0,60.0,4.0,6.0,61.0,3.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()